In [3]:
import pandas as pd
import numpy as np

In [4]:
splits = {'train': 'train.csv', 'test': 'test.csv'}
df = pd.read_csv("hf://datasets/agentlans/text-quality/" + splits["train"])

ImportError: Missing optional dependency 'fsspec'.  Use pip or conda to install fsspec.